# 0.   Packages and Functions

In [2]:
!pip install pandas_datareader

In [3]:

#general
import pandas as pd
import numpy as np
import pickle 
from decimal import Decimal


#for dates and times
import random
import time
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar

#prices
import pandas_datareader as web


In [4]:


def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)

In [5]:
def weekend2weekday (transactions, col_date, col_day):


  dates2_BUY=list()
  days2_BUY=list()

  for index,row in transactions.iterrows():
    buy_date = datetime.datetime.strptime(row[col_date], "%Y-%m-%d")
    if row[col_day]=='Sat':
      buy_date2 = buy_date - datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Fri'
      days2_BUY.append(buy_day2)
    elif row[col_day]=='Sun':
      buy_date = datetime.datetime.strptime(row[col_day], "%Y-%m-%d")
      buy_date2 = buy_date + datetime.timedelta(1)
      dates2_BUY.append(buy_date2)

      buy_day2='Mon'
      days2_BUY.append(buy_day2)
    else:
      buy_date2=row[col_date]
      dates2_BUY.append(buy_date2)
      buy_day2=row[col_day]
      days2_BUY.append(buy_day2)

    transactions['date_BUY_fix']=dates2_BUY
    transactions['day_BUY_fix']=days2_BUY


  return transactions

# 1. Setting investment parameters

## Temporal horizons
 In number of days
 - 1 day to 2 years

In [6]:
horizons=[1,2,3,4,5,6,7,10,15,20,30,40,50,60,70,80,90,100,120,150,180,210,240,270,300,330,360,420,480,540,600,660,720]


## Amounts in dollars
 - 50 to 50,000 dollars

In [7]:
amounts=[50,100,200,300,400,500,1000,1500,2000,3000,4000,5000,10000,15000,20000,25000,50000]

# 2. Data extraction

## Companies' stock prices

In [8]:
#Selecting a list of companies we are interested in analyzing:
companies = ['AAPL',
            'AMZN',
            'MSFT',
            'GOOG',
            'FB',
            'NVDA',
             #automobiles
            'F',
            'GM',
            'HMC',
            'TSLA',
            'TYO',
             'HOG',
             #retail
             'WMT',
             'M',
             'TGT',
             'KSS',
             'COST',
             'BBY',
             #FastMovingConsumerGoods(FMCG)
             'NESN',
             'KO',
             'PEP',
             'ULVR',
             'PG',
             'OR',
            #bank
            'AXP',
            'VISA',
            'SC',
            'DB',
            'UBS',
            'BAC',
            'JPM']

In [9]:
tech_c=['AAPL','AMZN','MSFT','GOOG','FB','VISA','NVDA']
auto_c=['F','GM','HMC','TSLA','TYO','HOG']
retail_c=[ 'WMT', 'M','TGT','KSS','COST','BBY']
fmcg_c=['NESN','KO','PEP','ULVR','PG', 'OR']
bank_c=['AXP','VISA','SC', 'DB', 'UBS','BAC','JPM']

In [10]:
#ESG ratios
ESG_ratios=[16.5, 27.4, 14.7, 22.8, 31.6, 12.9, 29.7, 30.6, 28.5, 31.1, 30.3, 16.3, 27.3, 14.5, 14.8, 12.9, 22.3, 12.0, 28.3, 25.1, 17.6, 23.4, 25.1, 20.5, 19.8, 17.4, 26.9, 30.0, 24.8, 26.3, 27.9]

We are going to extract info from 10 years from the companies selected

In [11]:
#Collecting the data of each company in a list
l_companies_prices=list()
error_companies=list()
error_indexes=list()

for company in companies:
    try:
        prices= web.DataReader(company, data_source='yahoo', start='2010-10-10', end='2020-10-10')
        l_companies_prices.append(prices)
        print(company)
    
    except :
        print('There were  errors when extracting data of  ', company)
        #elimino de la lista esa empresa que no se consiguieron datos.
        error_companies.append(company)
        
        error_company_index=companies.index(company)
        error_indexes.append(error_company_index)


for c in error_companies:
  companies.remove(c)

ESG_ratios_fix=list()

for index in range(0, len(ESG_ratios)):
    if index not in error_indexes:
        ESG_ratios_fix.append(ESG_ratios[index])

AAPL
AMZN
MSFT
GOOG
FB
NVDA
F
GM
HMC
TSLA
TYO
HOG
WMT
M
TGT
KSS
COST
BBY
There were  errors when extracting data of   NESN
KO
PEP
There were  errors when extracting data of   ULVR
PG
OR
AXP
There were  errors when extracting data of   VISA
SC
DB
UBS
BAC
JPM


In [12]:
companies

['AAPL',
 'AMZN',
 'MSFT',
 'GOOG',
 'FB',
 'NVDA',
 'F',
 'GM',
 'HMC',
 'TSLA',
 'TYO',
 'HOG',
 'WMT',
 'M',
 'TGT',
 'KSS',
 'COST',
 'BBY',
 'KO',
 'PEP',
 'PG',
 'OR',
 'AXP',
 'SC',
 'DB',
 'UBS',
 'BAC',
 'JPM']

In [13]:
len(ESG_ratios_fix)

28

In [14]:
len(companies)

28

In [15]:
print(companies[0])
l_companies_prices[0].columns

AAPL


Index(['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'], dtype='object')

In [16]:
company_pos=companies.index("AAPL")
l_companies_prices[company_pos].loc['2010-10-11','Close']

10.54857063293457

# 3. Random transactions

In [17]:
transactions=pd.DataFrame()
#we do 40,000 initial transactions

#id
transactions['id']=range(1,40001)

## Companies

In [18]:
#random company
transactions["company"]=np.random.choice(companies, size=len(transactions))

## Sector

In [19]:
#sector of the company

random_companies=transactions['company'].tolist()
random_sectors=list()
for comp in random_companies:
    if comp in tech_c:
        sector='TECH'
    elif comp in auto_c:
        sector='AUTO'
    elif comp in retail_c:
        sector='RETAIL'
    elif comp in fmcg_c:
        sector='FMCG'
    elif comp in bank_c:
        sector='BANK'
    
    random_sectors.append(sector)

transactions['sector']=random_sectors

## Horizons

In [20]:
#random horizon
transactions["horizon"] = np.random.choice(horizons, size=len(transactions))


## Investment Amounts

In [21]:
#random investment amount
transactions["amount"] = np.random.choice(amounts, size=len(transactions))


## Dates - BUY

In [22]:
#random buy date
start='2013-10-10'
end='2018-10-10'

dates_buy=list()

for i in range(0, transactions.shape[0]):
    r_date=random_date(start,end,random.random())
    dates_buy.append(r_date)


print(len(dates_buy))
transactions['date_BUY']=dates_buy

40000


### Fixing dates (BUY)

In [23]:
day_names=list()

for d in dates_buy:

  dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=dt.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_BUY']=day_names


In [25]:
dates2_BUY=list()
days2_BUY=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
  if row['day_BUY']=='Sat':
    buy_date2 = buy_date - datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Fri'
    days2_BUY.append(buy_day2)
  elif row['day_BUY']=='Sun':
    buy_date = datetime.datetime.strptime(row['date_BUY'], "%Y-%m-%d")
    buy_date2 = buy_date + datetime.timedelta(1)
    dates2_BUY.append(buy_date2)

    buy_day2='Mon'
    days2_BUY.append(buy_day2)
  else:
    buy_date2=buy_date
    dates2_BUY.append(buy_date2)
    buy_day2=row['day_BUY']
    days2_BUY.append(buy_day2)


transactions['date_BUY_fix']=dates2_BUY
transactions['day_BUY_fix']=days2_BUY

# transactions=weekend2weekday(transactions,'date_BUY','day_BUY' )

In [26]:
#checking for holidays (NYSE is closed)

cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_BUY:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_BUY']=l_holidays

In [27]:
transactions_h=transactions[transactions.holiday_BUY=='Holiday']

transactions=transactions[transactions.holiday_BUY!='Holiday']

In [28]:

l_volatilities = []
for company in companies:
    company_pos=companies.index(company)
    df_company_prices = pd.DataFrame(l_companies_prices[company_pos])
    log_ret = np.log(df_company_prices['Close'] / df_company_prices['Close'].shift(1))
    volatility = log_ret.rolling(window=252).std() * np.sqrt(252)
    df_company_prices.insert(2, "Volatility", volatility, True)
    l_volatilities.append(df_company_prices)

print(l_volatilities)

2  
2010-10-14   9213600.0  16.657616  
2010-10-15   6672800.0  16.754620  
...                ...        ...  
2020-10-05  14631000.0   6.060000  
2020-10-06  22708200.0   5.810000  
2020-10-07  17965500.0   5.960000  
2020-10-08  21298800.0   6.290000  
2020-10-09  19317400.0   6.230000  

[2518 rows x 7 columns],                   High         Low  Volatility        Open       Close  \
Date                                                                     
2010-10-11   55.000000   54.240002         NaN   54.299999   54.849998   
2010-10-12   54.910000   54.470001         NaN   54.869999   54.740002   
2010-10-13   55.080002   54.349998         NaN   54.689999   54.419998   
2010-10-14   54.619999   53.520000         NaN   54.459999   54.070000   
2010-10-15   54.590000   53.970001         NaN   54.400002   54.330002   
...                ...         ...         ...         ...         ...   
2020-10-05  162.309998  159.990005    0.391384  159.990005  161.490005   
2020-10-06  161.

In [29]:
l_returns = []
for company in companies:
    company_pos=companies.index(company)
    df_company_prices = pd.DataFrame(l_companies_prices[company_pos])
    returns = (df_company_prices['Close'] - df_company_prices['Close'].shift(1)) / df_company_prices['Close'].shift(1)
    df_company_prices.insert(2, "returns", returns, True)
    l_returns.append(df_company_prices)

In [30]:
l_returns

3.240002  51.970001  0.016052         NaN  52.810001  53.169998   
 ...               ...        ...       ...         ...        ...        ...   
 2020-10-05  20.750000  20.049999  0.023046    0.924426  20.330000  20.420000   
 2020-10-06  20.750000  19.360001 -0.045544    0.925006  20.750000  19.490000   
 2020-10-07  20.740000  19.830000  0.059005    0.927001  19.830000  20.639999   
 2020-10-08  21.900000  20.719999  0.060562    0.928778  20.870001  21.889999   
 2020-10-09  22.240000  21.430000 -0.016446    0.928351  22.000000  21.530001   
 
                Volume  Adj Close  
 Date                              
 2010-10-11  2392500.0  37.662571  
 2010-10-12  2589000.0  37.592255  
 2010-10-13  3001500.0  37.275810  
 2010-10-14  5694400.0  36.797657  
 2010-10-15  4613400.0  37.388332  
 ...               ...        ...  
 2020-10-05  6275500.0  20.420000  
 2020-10-06  7438600.0  19.490000  
 2020-10-07  6694300.0  20.639999  
 2020-10-08  8074000.0  21.889999  
 2020-10-09  

In [31]:
l_yearly_returns = []
for company in companies:
    company_pos=companies.index(company)
    df_company_prices = pd.DataFrame(l_companies_prices[company_pos])
    try:
        df_yearly = df_company_prices["Close"].resample('Y').first().pct_change()
        
        yearly_mean = df_yearly.mean()
    except:
        print( company)
        print(df_company_prices)
        yearly_mean = 0
    l_yearly_returns.append([company,yearly_mean])
print(l_yearly_returns)

[['AAPL', 0.24576212706845474], ['AMZN', 0.32878772362638664], ['MSFT', 0.21823741952413256], ['GOOG', 0.191378431933231], ['FB', 0.30161872761801845], ['NVDA', 0.4931171561846849], ['F', -0.011976915482311935], ['GM', 0.04088241949150044], ['HMC', -0.009170559827996482], ['TSLA', 0.5157369284319866], ['TYO', -0.10592633571589997], ['HOG', 0.044568829075491104], ['WMT', 0.09703322358290575], ['M', 0.008681139192578402], ['TGT', 0.11412148248236709], ['KSS', 0.006854575724160139], ['COST', 0.1674207351165724], ['BBY', 0.2622984487062406], ['KO', 0.06441514580167561], ['PEP', 0.07940352791222091], ['PG', 0.07725201668740694], ['OR', -0.06175039426047935], ['AXP', 0.14723187952081127], ['SC', 0.011489650389287712], ['DB', -0.14093138981130585], ['UBS', -0.0394967250395171], ['BAC', 0.18910217427914794], ['JPM', 0.15206171805688737]]


## Prices - BUY

In [32]:
#price when bought

l_buy_prices=list()
l_volatility=list()
for index,row in transactions.iterrows():
  comp=row['company']
  buy_date=row['date_BUY_fix']
  company_pos=companies.index(comp)
  try:
    price=l_companies_prices[company_pos].loc[buy_date,'Adj Close']
    volatility= l_volatilities[company_pos].loc[buy_date, 'Volatility']
  except :
    price=0
    

  l_buy_prices.append(price)

transactions['price_BUY']=l_buy_prices


In [33]:
l_volatility_buy=list()
for index,row in transactions.iterrows():
  comp=row['company']
  buy_date=row['date_BUY_fix']
  company_pos=companies.index(comp)
  try:
    volatility= l_volatilities[company_pos].loc[buy_date, 'Volatility']
  except :
    volatility=0
  l_volatility_buy.append(volatility)
print(len(l_volatility_buy))


transactions['Volatility_Buy'] = l_volatility_buy

38069


## Dates - SELL

In [34]:
#random sell date
dates_SELL=list()
expected_return = list()

for index,row in transactions.iterrows():
  comp=row['company']
  company_pos=companies.index(comp)
  buy_date = row['date_BUY_fix']
  sell_date = buy_date + datetime.timedelta(row['horizon'])
  expected_return.append(row['horizon']/365 * l_yearly_returns[company_pos][1])
  
  dates_SELL.append(sell_date)

transactions['expected_return (yearly)'] = expected_return
transactions['date_SELL']=dates_SELL

### Fixing dates (SELL)

In [35]:
day_names=list()



for d in dates_SELL:

  # dt = datetime.datetime.strptime(d, '%Y-%m-%d')
  weekno=d.weekday()

  if weekno == 0:
    dia='Mon'
    day_names.append(dia)
  elif weekno==1:
    dia='Tue'
    day_names.append(dia)
  elif weekno==2:
    dia='Wed'
    day_names.append(dia)
  elif weekno==3:
    dia='Thu'
    day_names.append(dia)
  elif weekno==4:
    dia='Fri'
    day_names.append(dia)
  elif weekno==5:
    dia='Sat'
    day_names.append(dia)
  elif weekno==6:
    dia='Sun'
    day_names.append(dia)

transactions['day_SELL']=day_names


In [36]:
dates2_SELL=list()
days2_SELL=list()

#if date is saturday, we are going to push it to friday
#if date is sunday, we are going to push it to monday

for index,row in transactions.iterrows():
  SELL_date = row['date_SELL']
  if row['day_SELL']=='Sat':
    SELL_date2 = SELL_date - datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Fri'
    days2_SELL.append(SELL_day2)
  elif row['day_SELL']=='Sun':
    SELL_date = row['date_SELL']
    SELL_date2 = SELL_date + datetime.timedelta(1)
    dates2_SELL.append(SELL_date2)

    SELL_day2='Mon'
    days2_SELL.append(SELL_day2)
  else:
    SELL_date2=SELL_date
    dates2_SELL.append(SELL_date2)
    SELL_day2=row['day_SELL']
    days2_SELL.append(SELL_day2)


transactions['date_SELL_fix']=dates2_SELL
transactions['day_SELL_fix']=days2_SELL

# transactions=weekend2weekday(transactions,'date_BUY','day_BUY' )

In [37]:
l_sharpe=list()
for index,row in transactions.iterrows():
  comp=row['company']
  buy_date=row['date_BUY_fix']
  company_pos=companies.index(comp)
  r = l_returns
  try:
    df_company_returns = pd.DataFrame(l_returns[company_pos])
    r=df_company_returns["returns"]
    sr = r.mean()/r.std() * np.sqrt(252)
    sr= l_volatilities[company_pos].loc[buy_date, 'Volatility']
  except :
    sr=0
  l_sharpe.append(sr)
print(len(l_sharpe))


transactions['Sharpe Ratio'] = l_sharpe

38069


In [38]:
transactions

,id,company,sector,horizon,amount,date_BUY,day_BUY,date_BUY_fix,day_BUY_fix,holiday_BUY,price_BUY,Volatility_Buy,expected_return (yearly),date_SELL,day_SELL,date_SELL_fix,day_SELL_fix,Sharpe Ratio
0,1,GM,AUTO,70,20000,2015-05-17,Sun,2015-05-18,Mon,,28.277824,0.229780,0.007840,2015-07-27,Mon,2015-07-27,Mon,0.229780
1,2,HMC,AUTO,150,400,2016-08-09,Tue,2016-08-09,Tue,,29.219999,0.270394,-0.003769,2017-01-06,Fri,2017-01-06,Fri,0.270394
2,3,PG,FMCG,80,50,2015-11-13,Fri,2015-11-13,Fri,,63.663868,0.156865,0.016932,2016-02-01,Mon,2016-02-01,Mon,0.156865
3,4,KSS,RETAIL,30,4000,2018-03-08,Thu,2018-03-08,Thu,,55.072594,0.348488,0.000563,2018-04-07,Sat,2018-04-06,Fri,0.348488
4,5,SC,BANK,270,1000,2014-05-13,Tue,2014-05-13,Tue,,19.326445,NaN,0.008499,2015-02-07,Sat,2015-02-06,Fri,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,TYO,AUTO,480,20000,2013-10-23,Wed,2013-10-23,Wed,,23.184040,0.172482,-0.139300,2015-02-15,Sun,2015-02-16,Mon,0.172482
39996,39997,NVDA,TECH,70,400,2017-04-02,Sun,2017-04-03,Mon,,107.198616,0.440058,0.094570,2017-06-12,Mon,2017-06-12,Mon,0.440058
39997,39998,F,AUTO,210,3000,2014-12-13,Sat,2014-12-12,Fri,,10.969175,0.215025,-0.006891,2015-07-10,Fri,2015-07-10,Fri,0.215025
39998,39999,TYO,AUTO,70,1500,2017-02-17,Fri,2017-02-17,Fri,,14.813115,0.191231,-0.020315,2017-04-28,Fri,2017-04-28,Fri,0.191231


In [39]:
#checking for holidays (NYSE, NASDAQ is closed)
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2010-10-10', end='2020-10-10').to_pydatetime()
# if datetime.datetime(2014,01,01) in holidays:
#     print True 

l_holidays=list()
for d in dates2_SELL:
  if d in holidays:
    holiday='Holiday'
    l_holidays.append(holiday)

  else:
    holiday=''
    l_holidays.append(holiday)

transactions['holiday_SELL']=l_holidays

## Prices - SELL

In [40]:
#price when sold

l_sell_prices=list()
for index,row in transactions.iterrows():
  comp=row['company']
  sell_date=row['date_SELL_fix']
  company_pos=companies.index(comp)
  try:
    price=l_companies_prices[company_pos].loc[sell_date,'Adj Close']
  except :
    price=0

  l_sell_prices.append(price)

transactions['price_SELL']=l_sell_prices


In [41]:
l_volatility_sell=list()
for index,row in transactions.iterrows():
  comp=row['company']
  sell_date=row['date_SELL_fix']
  company_pos=companies.index(comp)
  try:
    volatility=l_volatilities[company_pos].loc[sell_date,'Volatility']
  except :
    volatility=0

  l_volatility_sell.append(volatility)

transactions['Volatility_sell']=l_volatility_sell

In [42]:
transactions_h=transactions[transactions.holiday_SELL=='Holiday']

transactions=transactions[transactions.holiday_SELL!='Holiday']

In [43]:
transactions

,id,company,sector,horizon,amount,date_BUY,day_BUY,date_BUY_fix,day_BUY_fix,holiday_BUY,...,Volatility_Buy,expected_return (yearly),date_SELL,day_SELL,date_SELL_fix,day_SELL_fix,Sharpe Ratio,holiday_SELL,price_SELL,Volatility_sell
0,1,GM,AUTO,70,20000,2015-05-17,Sun,2015-05-18,Mon,,...,0.229780,0.007840,2015-07-27,Mon,2015-07-27,Mon,0.229780,,25.310125,0.232071
1,2,HMC,AUTO,150,400,2016-08-09,Tue,2016-08-09,Tue,,...,0.270394,-0.003769,2017-01-06,Fri,2017-01-06,Fri,0.270394,,30.469999,0.243365
2,3,PG,FMCG,80,50,2015-11-13,Fri,2015-11-13,Fri,,...,0.156865,0.016932,2016-02-01,Mon,2016-02-01,Mon,0.156865,,70.435738,0.164046
3,4,KSS,RETAIL,30,4000,2018-03-08,Thu,2018-03-08,Thu,,...,0.348488,0.000563,2018-04-07,Sat,2018-04-06,Fri,0.348488,,57.229309,0.349887
4,5,SC,BANK,270,1000,2014-05-13,Tue,2014-05-13,Tue,,...,NaN,0.008499,2015-02-07,Sat,2015-02-06,Fri,NaN,,18.577284,0.299217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39994,39995,OR,FMCG,7,200,2014-12-31,Wed,2014-12-31,Wed,,...,0.000000,-0.001184,2015-01-07,Wed,2015-01-07,Wed,0.000000,,0.000000,0.000000
39996,39997,NVDA,TECH,70,400,2017-04-02,Sun,2017-04-03,Mon,,...,0.440058,0.094570,2017-06-12,Mon,2017-06-12,Mon,0.440058,,148.491501,0.475060
39997,39998,F,AUTO,210,3000,2014-12-13,Sat,2014-12-12,Fri,,...,0.215025,-0.006891,2015-07-10,Fri,2015-07-10,Fri,0.215025,,10.805017,0.213415
39998,39999,TYO,AUTO,70,1500,2017-02-17,Fri,2017-02-17,Fri,,...,0.191231,-0.020315,2017-04-28,Fri,2017-04-28,Fri,0.191231,,14.020080,0.192017


In [44]:
transactions_ready=transactions[['company','sector','horizon','amount','date_BUY_fix','date_SELL_fix','price_BUY','price_SELL','Volatility_Buy','Volatility_sell', 'Sharpe Ratio', 'expected_return (yearly)']]

In [45]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly)
0,GM,AUTO,70,20000,2015-05-18,2015-07-27,28.277824,25.310125,0.229780,0.232071,0.229780,0.007840
1,HMC,AUTO,150,400,2016-08-09,2017-01-06,29.219999,30.469999,0.270394,0.243365,0.270394,-0.003769
2,PG,FMCG,80,50,2015-11-13,2016-02-01,63.663868,70.435738,0.156865,0.164046,0.156865,0.016932
3,KSS,RETAIL,30,4000,2018-03-08,2018-04-06,55.072594,57.229309,0.348488,0.349887,0.348488,0.000563
4,SC,BANK,270,1000,2014-05-13,2015-02-06,19.326445,18.577284,NaN,0.299217,NaN,0.008499
...,...,...,...,...,...,...,...,...,...,...,...,...
39994,OR,FMCG,7,200,2014-12-31,2015-01-07,0.000000,0.000000,0.000000,0.000000,0.000000,-0.001184
39996,NVDA,TECH,70,400,2017-04-03,2017-06-12,107.198616,148.491501,0.440058,0.475060,0.440058,0.094570
39997,F,AUTO,210,3000,2014-12-12,2015-07-10,10.969175,10.805017,0.215025,0.213415,0.215025,-0.006891
39998,TYO,AUTO,70,1500,2017-02-17,2017-04-28,14.813115,14.020080,0.191231,0.192017,0.191231,-0.020315


## Filtering transactions with error (Price = 0)

In [46]:
price_error=transactions_ready[(transactions_ready.price_BUY==0) | (transactions_ready.price_SELL==0)]
print('with errors: ', price_error.shape)
print(transactions_ready.shape)

transactions_ready=pd.concat([transactions_ready, price_error, price_error]).drop_duplicates(keep=False)
print(transactions_ready.shape)

with errors:  (1463, 12)
(36264, 12)
(34729, 12)


# Inflacion
Buscamos que supere la tasa de inflacion

In [134]:
# # inflation=pd.read_csv('ratios_inflacion.csv', index_col=False)

# inflation['Year'] = pd.to_datetime(inflation['Year'])

# inflation=inflation.set_index('Year')

# # inflation.info()

In [143]:
# l_inflation=list()
# for index,row in transactions.iterrows():
#     buy_date=row['date_BUY_fix']
#     year=buy_date.year
#     print(year)
#     try:
#         inflation=inflation.loc[year,'Tasa_inflacion']
#     except :
#         inflation=0

#     l_inflation.append(inflation)

# transactions['inflation']=l_inflation


2014
2016
2018
2016
2015
2016
2017
2015
2017
2017
2014
2018
2017
2016
2017
2015
2017
2016
2017
2015
2014
2014
2017
2017
2018
2014
2018
2018
2018
2017
2017
2016
2016
2014
2017
2014
2016
2016
2015
2014
2015
2014
2013
2014
2016
2018
2014
2015
2014
2016
2018
2017
2013
2017
2017
2016
2015
2018
2017
2013
2014
2015
2013
2015
2018
2016
2016
2018
2015
2016
2015
2017
2014
2018
2014
2016
2017
2015
2014
2014
2017
2017
2016
2014
2017
2016
2018
2015
2018
2017
2015
2018
2017
2018
2017
2018
2015
2015
2016
2015
2014
2016
2015
2015
2018
2016
2017
2016
2017
2015
2016
2016
2015
2015
2016
2016
2017
2014
2015
2015
2015
2016
2015
2014
2015
2015
2017
2016
2015
2017
2017
2013
2018
2017
2014
2014
2013
2015
2017
2014
2014
2016
2017
2015
2013
2018
2016
2015
2018
2016
2016
2014
2014
2018
2018
2014
2016
2015
2018
2017
2015
2014
2017
2014
2015
2014
2016
2016
2016
2018
2016
2014
2016
2018
2014
2014
2015
2014
2017
2013
2017
2018
2018
2015
2018
2018
2017
2017
2013
2014
2014
2017
2015
2017
2018
2015
2014
2014
2018
2016


2017
2016
2016
2014
2014
2018
2013
2018
2017
2018
2015
2014
2016
2015
2016
2015
2018
2015
2015
2014
2017
2017
2016
2017
2016
2018
2015
2018
2014
2016
2018
2015
2018
2017
2015
2014
2018
2015
2016
2014
2014
2016
2017
2016
2015
2014
2017
2013
2014
2018
2016
2018
2014
2013
2015
2017
2015
2017
2018
2017
2014
2013
2018
2016
2015
2018
2015
2016
2015
2017
2018
2014
2018
2016
2016
2016
2015
2014
2015
2014
2016
2014
2015
2015
2017
2014
2015
2015
2016
2014
2017
2014
2014
2015
2014
2017
2016
2014
2016
2016
2017
2016
2018
2018
2016
2016
2018
2015
2014
2017
2014
2018
2018
2015
2016
2016
2018
2015
2018
2015
2017
2018
2018
2014
2016
2017
2017
2017
2018
2014
2018
2014
2017
2016
2014
2018
2015
2018
2014
2016
2014
2018
2014
2017
2014
2015
2018
2018
2015
2016
2014
2015
2015
2017
2015
2018
2015
2014
2015
2018
2014
2015
2017
2015
2014
2016
2015
2016
2018
2015
2017
2015
2014
2017
2017
2016
2017
2015
2015
2018
2017
2017
2014
2014
2014
2015
2014
2014
2017
2018
2017
2015
2014
2015
2017
2017
2017
2016
2015
2014


2018
2015
2016
2016
2017
2015
2015
2016
2016
2017
2015
2018
2015
2016
2016
2016
2017
2017
2015
2014
2016
2015
2018
2014
2016
2016
2014
2015
2016
2015
2018
2014
2015
2015
2014
2015
2015
2018
2016
2013
2016
2017
2014
2014
2016
2013
2013
2016
2016
2016
2015
2013
2014
2017
2015
2018
2014
2014
2017
2014
2014
2018
2017
2014
2016
2018
2013
2015
2017
2016
2015
2018
2016
2017
2014
2014
2015
2016
2015
2018
2014
2018
2017
2016
2017
2017
2015
2014
2017
2015
2015
2014
2015
2016
2018
2016
2016
2018
2015
2016
2014
2016
2017
2014
2014
2015
2016
2016
2014
2015
2017
2016
2015
2016
2015
2014
2014
2017
2013
2014
2014
2014
2016
2017
2018
2018
2016
2014
2017
2017
2016
2014
2015
2015
2018
2017
2017
2017
2014
2014
2015
2015
2014
2014
2015
2015
2015
2016
2015
2017
2014
2015
2017
2015
2015
2015
2017
2018
2017
2013
2018
2018
2014
2018
2017
2017
2017
2015
2014
2017
2015
2015
2017
2015
2017
2018
2015
2018
2016
2015
2015
2017
2017
2014
2014
2016
2015
2014
2018
2017
2018
2017
2018
2018
2017
2018
2014
2014
2014
2015


2016
2014
2017
2015
2014
2015
2013
2016
2014
2013
2015
2015
2017
2014
2017
2018
2014
2016
2017
2018
2017
2016
2014
2014
2018
2017
2016
2016
2014
2016
2015
2017
2017
2016
2014
2015
2016
2017
2016
2013
2018
2013
2016
2018
2014
2017
2017
2016
2017
2017
2013
2016
2018
2015
2016
2016
2014
2015
2015
2016
2018
2015
2014
2016
2014
2015
2016
2014
2014
2017
2014
2016
2017
2016
2017
2018
2017
2014
2018
2018
2014
2015
2016
2014
2018
2015
2014
2016
2016
2015
2018
2018
2016
2015
2018
2015
2017
2014
2014
2014
2016
2017
2016
2017
2017
2015
2015
2014
2018
2016
2017
2016
2016
2018
2016
2016
2014
2016
2018
2017
2018
2014
2014
2015
2016
2015
2013
2015
2014
2013
2013
2013
2015
2014
2017
2017
2017
2015
2017
2018
2017
2013
2018
2017
2014
2018
2015
2017
2018
2015
2017
2017
2015
2018
2014
2015
2017
2015
2018
2018
2015
2018
2014
2015
2015
2017
2015
2015
2018
2017
2014
2016
2016
2014
2017
2017
2015
2017
2014
2017
2017
2018
2017
2017
2016
2016
2015
2015
2014
2017
2014
2015
2016
2016
2017
2015
2016
2017
2017
2018


2014
2015
2014
2016
2016
2014
2014
2016
2018
2016
2013
2014
2016
2014
2013
2013
2014
2015
2015
2015
2015
2018
2015
2015
2016
2016
2016
2018
2016
2016
2018
2013
2017
2015
2017
2014
2014
2015
2016
2017
2018
2015
2017
2015
2017
2018
2016
2014
2017
2015
2015
2014
2015
2014
2016
2014
2014
2017
2014
2017
2014
2015
2015
2017
2016
2014
2015
2014
2014
2015
2017
2018
2014
2014
2015
2016
2017
2017
2014
2015
2018
2018
2014
2018
2015
2017
2015
2017
2014
2018
2017
2016
2016
2015
2017
2017
2017
2018
2016
2017
2018
2018
2017
2014
2016
2017
2015
2014
2017
2016
2014
2017
2014
2014
2015
2015
2017
2018
2016
2018
2015
2016
2016
2016
2016
2014
2015
2016
2015
2015
2017
2016
2014
2016
2016
2013
2015
2018
2015
2014
2015
2017
2015
2016
2014
2013
2015
2014
2016
2016
2015
2015
2014
2014
2017
2018
2015
2015
2014
2018
2013
2016
2013
2014
2018
2016
2017
2017
2016
2014
2016
2017
2015
2018
2013
2016
2015
2014
2016
2017
2016
2017
2014
2016
2018
2017
2016
2016
2018
2015
2014
2015
2014
2014
2017
2015
2016
2014
2017
2018


...

2018
2015
2017
2016
2016
2017
2017
2014
2015
2018
2014
2014
2014
2014
2017
2013
2014
2014
2016
2014
2016
2018
2017
2017
2016
2018
2015
2016
2014
2018
2018
2015
2016
2017
2014
2014
2018
2017
2014
2017
2017
2017
2015
2016
2014
2014
2017
2014
2016
2016
2017
2016
2013
2018
2018
2013
2015
2014
2015
2014
2018
2018
2018
2014
2014
2018
2017
2015
2014
2018
2017
2018
2016
2017
2017
2017
2015
2018
2016
2015
2014
2017
2017
2015
2014
2016
2018
2014
2014
2018
2015
2015
2016
2013
2016
2013
2017
2017
2017
2018
2016
2017
2014
2017
2017
2018
2016
2016
2018
2015
2014
2015
2017
2017
2014
2014
2017
2017
2017
2015
2014
2015
2015
2018
2016
2018
2017
2016
2014
2014
2014
2016
2018
2017
2014
2015
2016
2016
2018
2017
2016
2015
2013
2014
2017
2015
2014
2017
2017
2016
2015
2017
2013
2015
2016
2014
2014
2014
2015
2017
2017
2015
2017
2018
2014
2015
2018
2013
2018
2016
2014
2018
2016
2016
2018
2016
2017
2017
2016
2014
2018
2018
2017
2014
2018
2014
2017
2018
2016
2015
2015
2017
2017
2015
2016
2016
2015
2015
2016
2014


2017
2018
2016
2014
2014
2014
2014
2013
2016
2014
2013
2015
2014
2014
2014
2017
2014
2016
2017
2014
2013
2015
2018
2014
2017
2017
2016
2016
2016
2014
2015
2016
2017
2016
2016
2016
2014
2018
2018
2018
2016
2017
2014
2015
2014
2017
2017
2017
2015
2016
2017
2017
2016
2017
2014
2015
2016
2017
2017
2014
2017
2016
2017
2014
2016
2018
2014
2016
2015
2015
2015
2016
2014
2017
2016
2015
2014
2015
2014
2018
2017
2016
2016
2014
2016
2016
2016
2016
2017
2017
2017
2018
2017
2017
2016
2015
2017
2017
2016
2015
2014
2013
2014
2014
2015
2014
2014
2017
2017
2017
2015
2015
2014
2018
2018
2015
2016
2015
2014
2015
2018
2014
2017
2015
2014
2016
2015
2018
2017
2018
2018
2013
2018
2015
2013
2018
2016
2015
2014
2016
2016
2016
2013
2018
2014
2016
2017
2016
2014
2015
2016
2018
2017
2015
2014
2015
2018
2015
2015
2018
2014
2014
2017
2018
2014
2014
2018
2015
2015
2016
2017
2018
2014
2015
2014
2017
2016
2015
2017
2014
2016
2017
2015
2014
2016
2016
2015
2014
2018
2018
2015
2016
2016
2014
2016
2018
2015
2016
2014
2014


2014
2015
2017
2017
2016
2018
2014
2018
2016
2015
2015
2014
2017
2014
2017
2016
2017
2014
2017
2016
2016
2014
2015
2015
2015
2017
2016
2014
2018
2015
2016
2016
2014
2018
2014
2017
2017
2018
2014
2018
2018
2014
2015
2014
2018
2015
2018
2015
2014
2014
2018
2018
2014
2014
2013
2013
2014
2015
2014
2016
2018
2016
2015
2014
2016
2018
2013
2017
2017
2015
2018
2015
2015
2014
2018
2013
2017
2016
2017
2015
2015
2018
2014
2017
2014
2015
2017
2014
2014
2016
2014
2015
2017
2017
2015
2017
2014
2015
2018
2016
2017
2018
2018
2018
2016
2018
2015
2015
2015
2018
2016
2017
2015
2017
2016
2015
2016
2018
2015
2015
2016
2017
2013
2014
2017
2016
2017
2015
2016
2017
2018
2016
2018
2015
2017
2016
2017
2016
2017
2015
2017
2015
2014
2018
2015
2016
2018
2017
2014
2018
2016
2017
2017
2017
2013
2015
2014
2015
2014
2018
2017
2014
2016
2015
2017
2015
2016
2017
2014
2014
2017
2014
2016
2018
2014
2016
2013
2018
2018
2018
2014
2014
2016
2016
2017
2016
2016
2018
2018
2018
2014
2015
2014
2015
2017
2016
2016
2017
2014
2017


2015
2017
2015
2017
2016
2014
2015
2013
2014
2017
2015
2014
2017
2016
2017
2018
2017
2015
2017
2014
2016
2016
2014
2018
2015
2014
2016
2018
2015
2017
2014
2015
2017
2014
2014
2018
2014
2014
2016
2016
2017
2016
2015
2017
2016
2018
2016
2015
2015
2016
2014
2016
2014
2017
2017
2014
2014
2014
2016
2017
2014
2014
2014
2015
2016
2015
2018
2017
2014
2015
2014
2018
2017
2014
2017
2017
2014
2014
2015
2017
2016
2014
2017
2014
2015
2014
2017
2016
2016
2015
2017
2015
2016
2015
2017
2016
2017
2016
2015
2017
2015
2014
2017
2014
2016
2018
2015
2015
2014
2018
2017
2015
2016
2016
2016
2014
2016
2018
2017
2018
2016
2016
2014
2014
2016
2016
2015
2015
2016
2015
2016
2015
2018
2015
2015
2017
2017
2014
2015
2015
2015
2014
2014
2015
2018
2017
2016
2016
2016
2016
2017
2017
2013
2017
2017
2014
2014
2018
2014
2013
2013
2013
2016
2018
2015
2016
2018
2017
2018
2015
2018
2015
2014
2013
2017
2014
2016
2017
2017
2014
2017
2018
2016
2014
2016
2014
2014
2017
2016
2013
2016
2016
2013
2016
2013
2016
2015
2016
2014
2014


<ipython-input-143-20dbdd0dd96e>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions['inflation']=l_inflation


# Real Profit/Loss (Rentabilidad Real)

In [47]:
transactions_ready['nominal_return']=(transactions_ready['price_SELL']-transactions_ready['price_BUY'])/transactions_ready['price_BUY']

In [48]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly),nominal_return
0,GM,AUTO,70,20000,2015-05-18,2015-07-27,28.277824,25.310125,0.229780,0.232071,0.229780,0.007840,-0.104948
1,HMC,AUTO,150,400,2016-08-09,2017-01-06,29.219999,30.469999,0.270394,0.243365,0.270394,-0.003769,0.042779
2,PG,FMCG,80,50,2015-11-13,2016-02-01,63.663868,70.435738,0.156865,0.164046,0.156865,0.016932,0.106369
3,KSS,RETAIL,30,4000,2018-03-08,2018-04-06,55.072594,57.229309,0.348488,0.349887,0.348488,0.000563,0.039161
4,SC,BANK,270,1000,2014-05-13,2015-02-06,19.326445,18.577284,NaN,0.299217,NaN,0.008499,-0.038764
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39993,F,AUTO,180,25000,2017-07-31,2018-01-26,9.533318,10.021590,0.205644,0.183368,0.205644,-0.005906,0.051217
39996,NVDA,TECH,70,400,2017-04-03,2017-06-12,107.198616,148.491501,0.440058,0.475060,0.440058,0.094570,0.385200
39997,F,AUTO,210,3000,2014-12-12,2015-07-10,10.969175,10.805017,0.215025,0.213415,0.215025,-0.006891,-0.014965
39998,TYO,AUTO,70,1500,2017-02-17,2017-04-28,14.813115,14.020080,0.191231,0.192017,0.191231,-0.020315,-0.053536


In [49]:
transactions_ready['investment']=''
investments=list()
for index, row in transactions_ready.iterrows():
  if row['nominal_return']>0:
    inv='GOOD'
    investments.append(inv)
  elif row['nominal_return']<0:
    inv='BAD'
    investments.append(inv)
  else:
    inv='BAD'
    investments.append(inv)

transactions_ready['investment']=investments

In [50]:
transactions_ready.shape

(34729, 14)

In [51]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly),nominal_return,investment
0,GM,AUTO,70,20000,2015-05-18,2015-07-27,28.277824,25.310125,0.229780,0.232071,0.229780,0.007840,-0.104948,BAD
1,HMC,AUTO,150,400,2016-08-09,2017-01-06,29.219999,30.469999,0.270394,0.243365,0.270394,-0.003769,0.042779,GOOD
2,PG,FMCG,80,50,2015-11-13,2016-02-01,63.663868,70.435738,0.156865,0.164046,0.156865,0.016932,0.106369,GOOD
3,KSS,RETAIL,30,4000,2018-03-08,2018-04-06,55.072594,57.229309,0.348488,0.349887,0.348488,0.000563,0.039161,GOOD
4,SC,BANK,270,1000,2014-05-13,2015-02-06,19.326445,18.577284,NaN,0.299217,NaN,0.008499,-0.038764,BAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39993,F,AUTO,180,25000,2017-07-31,2018-01-26,9.533318,10.021590,0.205644,0.183368,0.205644,-0.005906,0.051217,GOOD
39996,NVDA,TECH,70,400,2017-04-03,2017-06-12,107.198616,148.491501,0.440058,0.475060,0.440058,0.094570,0.385200,GOOD
39997,F,AUTO,210,3000,2014-12-12,2015-07-10,10.969175,10.805017,0.215025,0.213415,0.215025,-0.006891,-0.014965,BAD
39998,TYO,AUTO,70,1500,2017-02-17,2017-04-28,14.813115,14.020080,0.191231,0.192017,0.191231,-0.020315,-0.053536,BAD


In [52]:
transactions_ready.investment.value_counts()

GOOD    20137
BAD     14592
Name: investment, dtype: int64

# Ratios

In [53]:
l_PE_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix'].year
    company=row['company']
    try:
        PE_ratio=PE_ratios.loc[year,company]
    except:
        PE_ratio=0
    
    l_PE_ratios.append(PE_ratio)
    

In [54]:
transactions_ready['PE_ratio']=l_PE_ratios

In [55]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly),nominal_return,investment,PE_ratio
0,GM,AUTO,70,20000,2015-05-18,2015-07-27,28.277824,25.310125,0.229780,0.232071,0.229780,0.007840,-0.104948,BAD,0
1,HMC,AUTO,150,400,2016-08-09,2017-01-06,29.219999,30.469999,0.270394,0.243365,0.270394,-0.003769,0.042779,GOOD,0
2,PG,FMCG,80,50,2015-11-13,2016-02-01,63.663868,70.435738,0.156865,0.164046,0.156865,0.016932,0.106369,GOOD,0
3,KSS,RETAIL,30,4000,2018-03-08,2018-04-06,55.072594,57.229309,0.348488,0.349887,0.348488,0.000563,0.039161,GOOD,0
4,SC,BANK,270,1000,2014-05-13,2015-02-06,19.326445,18.577284,NaN,0.299217,NaN,0.008499,-0.038764,BAD,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39993,F,AUTO,180,25000,2017-07-31,2018-01-26,9.533318,10.021590,0.205644,0.183368,0.205644,-0.005906,0.051217,GOOD,0
39996,NVDA,TECH,70,400,2017-04-03,2017-06-12,107.198616,148.491501,0.440058,0.475060,0.440058,0.094570,0.385200,GOOD,0
39997,F,AUTO,210,3000,2014-12-12,2015-07-10,10.969175,10.805017,0.215025,0.213415,0.215025,-0.006891,-0.014965,BAD,0
39998,TYO,AUTO,70,1500,2017-02-17,2017-04-28,14.813115,14.020080,0.191231,0.192017,0.191231,-0.020315,-0.053536,BAD,0


## ESG Ratios

In [56]:
len(ESG_ratios_fix)

28

In [57]:
companies[3]

'GOOG'

In [58]:
ESG_ratios_fix

[16.5,
 27.4,
 14.7,
 22.8,
 31.6,
 12.9,
 29.7,
 30.6,
 28.5,
 31.1,
 30.3,
 16.3,
 27.3,
 14.5,
 14.8,
 12.9,
 22.3,
 12.0,
 25.1,
 17.6,
 25.1,
 20.5,
 19.8,
 26.9,
 30.0,
 24.8,
 26.3,
 27.9]

In [59]:
transactions_ready['ESG_ranking']=0

for index, row in transactions_ready.iterrows():
    company=row['company']
    ind_comp=companies.index(company)
    ESG_ratio= ESG_ratios_fix[ind_comp]
    transactions_ready.loc[index, 'ESG_ranking']=ESG_ratio
            

In [60]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly),nominal_return,investment,PE_ratio,ESG_ranking
0,GM,AUTO,70,20000,2015-05-18,2015-07-27,28.277824,25.310125,0.229780,0.232071,0.229780,0.007840,-0.104948,BAD,0,30.6
1,HMC,AUTO,150,400,2016-08-09,2017-01-06,29.219999,30.469999,0.270394,0.243365,0.270394,-0.003769,0.042779,GOOD,0,28.5
2,PG,FMCG,80,50,2015-11-13,2016-02-01,63.663868,70.435738,0.156865,0.164046,0.156865,0.016932,0.106369,GOOD,0,25.1
3,KSS,RETAIL,30,4000,2018-03-08,2018-04-06,55.072594,57.229309,0.348488,0.349887,0.348488,0.000563,0.039161,GOOD,0,12.9
4,SC,BANK,270,1000,2014-05-13,2015-02-06,19.326445,18.577284,NaN,0.299217,NaN,0.008499,-0.038764,BAD,0,26.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39993,F,AUTO,180,25000,2017-07-31,2018-01-26,9.533318,10.021590,0.205644,0.183368,0.205644,-0.005906,0.051217,GOOD,0,29.7
39996,NVDA,TECH,70,400,2017-04-03,2017-06-12,107.198616,148.491501,0.440058,0.475060,0.440058,0.094570,0.385200,GOOD,0,12.9
39997,F,AUTO,210,3000,2014-12-12,2015-07-10,10.969175,10.805017,0.215025,0.213415,0.215025,-0.006891,-0.014965,BAD,0,29.7
39998,TYO,AUTO,70,1500,2017-02-17,2017-04-28,14.813115,14.020080,0.191231,0.192017,0.191231,-0.020315,-0.053536,BAD,0,30.3


In [61]:
transactions_ready.loc[4,'date_BUY_fix']

Timestamp('2014-05-13 00:00:00')

In [63]:
ratios_collection= pickle.load(open('ratios_collection.p', "rb" ))

In [64]:
valor=ratios_collection['AAPL'][ratios_collection['AAPL']['Date']=='2020-09'].PE_Ratio.values[0]
float(valor)

35.46

In [65]:
#getting nearest date before investment.
def nearest(items, pivot):
    return min([i for i in items if i < pivot], key=lambda x: abs(x - pivot))

In [66]:
nearest(['Date'].dropna(),transactions_ready.loc[4,'date_BUY_fix'])

NameError: name 'df' is not defined

In [70]:
l_PE_ratios=list()
for index, row in transactions_ready.iterrows():
    year=row['date_BUY_fix']
    company=row['company']
    
    try:
        ratios_company=ratios_collection[company]
        date=nearest(ratios_company['Date'].dropna(),row['date_BUY_fix'])
    
        try:
            PE_ratio_str=ratios_collection[company][ratios_collection[company]['Date']==date].PE_Ratio.values[0]
            PE_ratio=float(PE_ratio_str)
        except:
            #se puede asignar un cero si hace falta.
            PE_ratio= 0
            
    except:
        PE_ratio=0
    
    l_PE_ratios.append(PE_ratio)

In [71]:
transactions_ready['PE_ratio']=l_PE_ratios

In [72]:
transactions_ready

,company,sector,horizon,amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly),nominal_return,investment,PE_ratio,ESG_ranking
0,GM,AUTO,70,20000,2015-05-18,2015-07-27,28.277824,25.310125,0.229780,0.232071,0.229780,0.007840,-0.104948,BAD,14.14,30.6
1,HMC,AUTO,150,400,2016-08-09,2017-01-06,29.219999,30.469999,0.270394,0.243365,0.270394,-0.003769,0.042779,GOOD,13.50,28.5
2,PG,FMCG,80,50,2015-11-13,2016-02-01,63.663868,70.435738,0.156865,0.164046,0.156865,0.016932,0.106369,GOOD,23.06,25.1
3,KSS,RETAIL,30,4000,2018-03-08,2018-04-06,55.072594,57.229309,0.348488,0.349887,0.348488,0.000563,0.039161,GOOD,11.12,12.9
4,SC,BANK,270,1000,2014-05-13,2015-02-06,19.326445,18.577284,NaN,0.299217,NaN,0.008499,-0.038764,BAD,0.00,26.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39993,F,AUTO,180,25000,2017-07-31,2018-01-26,9.533318,10.021590,0.205644,0.183368,0.205644,-0.005906,0.051217,GOOD,9.88,29.7
39996,NVDA,TECH,70,400,2017-04-03,2017-06-12,107.198616,148.491501,0.440058,0.475060,0.440058,0.094570,0.385200,GOOD,34.31,12.9
39997,F,AUTO,210,3000,2014-12-12,2015-07-10,10.969175,10.805017,0.215025,0.213415,0.215025,-0.006891,-0.014965,BAD,14.54,29.7
39998,TYO,AUTO,70,1500,2017-02-17,2017-04-28,14.813115,14.020080,0.191231,0.192017,0.191231,-0.020315,-0.053536,BAD,0.00,30.3


In [73]:
df_to_ml = transactions_ready[['investment','company','sector','horizon','amount','Volatility_Buy','Volatility_sell', 'Sharpe Ratio', 'expected_return (yearly)', 'PE_ratio', 'ESG_ranking']]

In [74]:
df_to_ml

,investment,company,sector,horizon,amount,Volatility_Buy,Volatility_sell,Sharpe Ratio,expected_return (yearly),PE_ratio,ESG_ranking
0,BAD,GM,AUTO,70,20000,0.229780,0.232071,0.229780,0.007840,14.14,30.6
1,GOOD,HMC,AUTO,150,400,0.270394,0.243365,0.270394,-0.003769,13.50,28.5
2,GOOD,PG,FMCG,80,50,0.156865,0.164046,0.156865,0.016932,23.06,25.1
3,GOOD,KSS,RETAIL,30,4000,0.348488,0.349887,0.348488,0.000563,11.12,12.9
4,BAD,SC,BANK,270,1000,NaN,0.299217,NaN,0.008499,0.00,26.9
...,...,...,...,...,...,...,...,...,...,...,...
39993,GOOD,F,AUTO,180,25000,0.205644,0.183368,0.205644,-0.005906,9.88,29.7
39996,GOOD,NVDA,TECH,70,400,0.440058,0.475060,0.440058,0.094570,34.31,12.9
39997,BAD,F,AUTO,210,3000,0.215025,0.213415,0.215025,-0.006891,14.54,29.7
39998,BAD,TYO,AUTO,70,1500,0.191231,0.192017,0.191231,-0.020315,0.00,30.3


In [75]:
df_to_ml.to_csv('df_to_ml.csv')